In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # pyplot is for showing images


In [ ]:
data=pd.read_csv('/kaggle/input/digit-recognizer/train.csv')#reads the file

In [ ]:
data=np.array(data)#converting data to numpy array
m,n=data.shape#m are rows and n are columns
np.random.shuffle(data)
#Development set
dev_data=data[0:1000].T#transposing 1000 examples
y_devdata=dev_data[0]#label
x_devdata=dev_data[1:n]#actual data set 

#Training sets, remaining examples after 1000
train_data=data[1000:m].T#transposing the rest of examples
y_train=train_data[0]#labels, the first column
x_train=train_data[1:n]#actual data set
x_train=x_train/255
_,m_train=x_train.shape

In [ ]:
#Parameters such as weights and biases
def init_parameters():
    w1=np.random.rand(10, 784) -0.5
    b1=np.random.rand(10,1)-0.5
    w2=np.random.rand(10,10)-0.5
    b2=np.random.rand(10,1)-0.5
    return w1,b1,w2,b2
    
def ReLU(z):#activation method for 0 to 1st layer
    return np.maximum(0,z)


def softmax(z):#activation method 1st to 2nd
    A = np.exp(z)/sum(np.exp(z))#collapses all rows(examples) into one and summed across all the columns which we divide with to each element which gives us the probability
    return A

#forward propogation
def forward_propogation(w1,b1,w2,b2,x):
    z1=w1.dot(x)+b1
    A1=ReLU(z1)
    z2=w2.dot(A1)+b2
    A2=softmax(z2)
    return z1,A1,z2,A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def derivative_ReLU(z):
    return z>0 #When z > 0: ReLU(z) = z, so derivative = 1

#backwards propogation to find error
def back_propogation(z1,A1,z2,A2,w1,w2,y,x):
    encoded_y=one_hot(y)
    dz2=A2-encoded_y
    dw2=1/m*dz2.dot(A1.T)
    db2=1/m*np.sum(dz2)
    dz1=w2.T.dot(dz2)*derivative_ReLU(z1)
    dw1=1/m*dz1.dot(x.T)
    db1=1/m*np.sum(dz1)
    return dw1, db1, dw2, db2


#updates parameters
def updata_param(w1,b1,w2,b2,dw2,db2,dw1,db1,alpha):
    w1=w1-dw1*alpha #alpha is the learning rate of the training
    b1=b1-db1*alpha
    w2=w2-dw2*alpha 
    b2=b2-db2*alpha
    return w1,b1,w2,b2


    





In [ ]:
def get_predictions(A2):
    return np.argmax(A2,0)
    

                     
def accuracy(predictions,y):
    print(predictions,y)
    return np.sum(predictions==y)/y.size


def gradient_desc(x, y, iterations, alpha):
    w1, b1, w2, b2 = init_parameters()
    for i in range(iterations):
        z1, A1, z2, A2 = forward_propogation(w1, b1, w2, b2, x)
        dw1, db1, dw2, db2 = back_propogation(z1,A1,z2,A2,w1,w2,y,x)
        w1, b1, w2, b2 = updata_param(w1, b1, w2, b2, dw2, db2, dw1, db1, alpha)
        if(i % 10 == 0):
            print("iteration:", i)
            print("Accuracy:", accuracy(get_predictions(A2), y))        
    return w1, b1, w2, b2

In [ ]:
w1,b1,w2,b2 =gradient_desc(x_train,y_train,500,0.10)

In [ ]:
def make_predictions(x, w1, b1, w2, b2):
    _, _, _, A2 = forward_propogation(w1, b1, w2, b2,x)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, w1, b1, w2, b2):
    current_image = x_train[:, index, None]
    prediction = make_predictions(x_train[:, index, None], w1, b1, w2, b2)
    label = y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
test_prediction(1042, w1, b1, w2, b2)
